In [21]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [22]:
from pandas import DataFrame
import json

In [23]:
from lyricsgenius import Genius
token = 'utWUfzSpqx5D-0h6kdlftC_wWX9s2iPsMP5T5QP0phbGprepCLAjRBZP3cmnhZhB'
genius = Genius(token, sleep_time=2)

In [24]:
from utils import ArtistSongs, Artists, Charts, Songs, Lyrics
from utils.Charts import Chart, ChartList
Artists.genius = genius
ArtistSongs.genius = genius
Charts.genius = genius
Songs.genius = genius
Lyrics.genius = genius

In [ ]:
Charts.chart_download()
ArtistSongs.load_from_charts()
Lyrics.load_from_artist_songs()

No more charts
Name: Drake; ID: 130
Can't add song by Drake & Future, artist must be Drake.
Can't add song by Lil Baby & Drake, artist must be Drake.
Can't add song by Drake, Kanye West, Lil Wayne & Eminem, artist must be Drake.
Can't add song by Nicki Minaj, artist must be Drake.
Name: Genius English Translations; ID: 196943
Name: Genius Romanizations; ID: 1507735
Name: Eminem; ID: 45
Can't add song by Drake, Kanye West, Lil Wayne & Eminem, artist must be Eminem.
Can't add song by 50 Cent, artist must be Eminem.
Name: Kanye West; ID: 72
Can't add song by JAY-Z & Kanye West, artist must be Kanye West.
Can't add song by Desiigner, artist must be Kanye West.
Can't add song by Kanye West, JAY-Z & Big Sean, artist must be Kanye West.
Can't add song by Drake, Kanye West, Lil Wayne & Eminem, artist must be Kanye West.
Can't add song by Kanye West, Chief Keef, Pusha T, Big Sean & Jadakiss, artist must be Kanye West.
Can't add song by Lil Nas X & Jack Harlow, artist must be Kanye West.
Can't a

Name: Doja Cat; ID: 139478
Can't add song by Doja Cat & The Weeknd, artist must be Doja Cat.
Can't add song by Ariana Grande & Doja Cat, artist must be Doja Cat.
Juicy already in Doja Cat, not adding song.
Name: Genius Türkçe Çeviri; ID: 569121
Name: Adele; ID: 2300
Can't add song by Sia, artist must be Adele.
Can't add song by Lil Wayne, artist must be Adele.
Chasing Pavements already in Adele, not adding song.
Can't add song by Conor Maynard, artist must be Adele.
Name: Kid Cudi; ID: 68
Can't add song by Kanye West, artist must be Kid Cudi.
Can't add song by Kanye West, artist must be Kid Cudi.
Can't add song by JAY-Z & Kanye West, artist must be Kid Cudi.
Can't add song by THE SCOTTS, Travis Scott & Kid Cudi, artist must be Kid Cudi.
Can't add song by KIDS SEE GHOSTS, artist must be Kid Cudi.
Can't add song by Kanye West, artist must be Kid Cudi.
Can't add song by Travis Scott, artist must be Kid Cudi.
Can't add song by JAY-Z & Kanye West, artist must be Kid Cudi.
Can't add song by 

Can't add song by Machine Gun Kelly & Camila Cabello, artist must be Machine Gun Kelly.
Can't add song by Machine Gun Kelly, X Ambassadors & Bebe Rexha, artist must be Machine Gun Kelly.
Can't add song by Machine Gun Kelly, YUNGBLUD & Travis Barker, artist must be Machine Gun Kelly.
Can't add song by Machine Gun Kelly & blackbear, artist must be Machine Gun Kelly.
Can't add song by Machine Gun Kelly & WILLOW, artist must be Machine Gun Kelly.
Can't add song by Machine Gun Kelly & Bring Me The Horizon, artist must be Machine Gun Kelly.
Name: Jhené Aiko; ID: 1020
Can't add song by Drake, artist must be Jhené Aiko.
Can't add song by TWENTY88, artist must be Jhené Aiko.
Can't add song by Big Sean, artist must be Jhené Aiko.
Can't add song by TWENTY88, artist must be Jhené Aiko.
Can't add song by Big Sean, artist must be Jhené Aiko.
Can't add song by TWENTY88, artist must be Jhené Aiko.
Name: Pink Floyd; ID: 694
Name: Coldplay; ID: 8351
Can't add song by The Chainsmokers & Coldplay, artist 

Can't add song by Lil Dicky, artist must be Miley Cyrus.
Can't add song by Ariana Grande, Miley Cyrus & Lana Del Rey, artist must be Miley Cyrus.
Can't add song by Ashley O, artist must be Miley Cyrus.
Can't add song by ​​alt-J, artist must be Miley Cyrus.
Can't add song by Artists For Haiti, artist must be Miley Cyrus.
Can't add song by Big Sean, artist must be Miley Cyrus.
Can't add song by Hannah Montana, artist must be Miley Cyrus.
Can't add song by Hannah Montana, artist must be Miley Cyrus.
Name: ScHoolboy Q; ID: 11353
Can't add song by ScHoolboy Q, 2 Chainz & Saudi, artist must be ScHoolboy Q.
Can't add song by A$AP Rocky, artist must be ScHoolboy Q.
Can't add song by Tyler, The Creator, artist must be ScHoolboy Q.
Can't add song by Kendrick Lamar, artist must be ScHoolboy Q.
Name: JAY-Z & Kanye West; ID: 652275
Can't add song by Marnik, Naeleck & VINAI, artist must be JAY-Z & Kanye West.
Can't add song by Young Hav, artist must be JAY-Z & Kanye West.
Name: Damso; ID: 45855
Name

Name: Fall Out Boy; ID: 7811
Name: Pusha T; ID: 643
Can't add song by Kanye West, Chief Keef, Pusha T, Big Sean & Jadakiss, artist must be Pusha T.
Can't add song by Kanye West, Pusha T & Ghostface Killah, artist must be Pusha T.
Can't add song by Kanye West, artist must be Pusha T.
Can't add song by Kanye West, artist must be Pusha T.
Can't add song by Kanye West, artist must be Pusha T.
Can't add song by Kanye West, artist must be Pusha T.
Can't add song by Kanye West & André 3000, artist must be Pusha T.
Can't add song by Metro Boomin, Future & Chris Brown, artist must be Pusha T.
Name: Kollegah; ID: 12077
Can't add song by Kollegah & Farid Bang, artist must be Kollegah.
Can't add song by Kollegah & Farid Bang, artist must be Kollegah.
Can't add song by Kollegah & Farid Bang, artist must be Kollegah.
Can't add song by Kollegah & Farid Bang, artist must be Kollegah.
Can't add song by Kollegah & Farid Bang, artist must be Kollegah.
Can't add song by Kollegah & Farid Bang, artist must 

...

Can't add song by Nicki Minaj & G Herbo, artist must be G Herbo.
Can't add song by Tink & G Herbo, artist must be G Herbo.
Can't add song by G Herbo & Southside, artist must be G Herbo.
Can't add song by G Herbo & Southside, artist must be G Herbo.
Can't add song by Juice WRLD, artist must be G Herbo.
Can't add song by G Herbo & Southside, artist must be G Herbo.
Can't add song by G Herbo & Southside, artist must be G Herbo.
id: 91049, artist:
{'artist': {'alternate_names': ['Wunna', 'Gunna Wunna', 'YSL Gunna', 'Sergio Giavanni Kitchens', 'Sergio Kitchens'], 'api_path': '/artists/91049', 'description': {'plain': "Sergio Giavanni Kitchens (born June 14, 1993), better known by his stage name Gunna, is an American rapper from College Park, Georgia. Known for his melodic flows, he was introduced to  Young Thug through friend, and mentor, Keith Troup and subsequently signed to Thug’s record label, Young Stoner Life Records.\n\nGunna dropped his debut mixtape Drip Season in October of 2016. 

Can't add song by Lil Baby & Gunna, artist must be Gunna.
Can't add song by Lil Baby, Gunna & Drake, artist must be Gunna.
Can't add song by Lil Baby & Gunna, artist must be Gunna.
Can't add song by Young Stoner Life, Young Thug & Gunna, artist must be Gunna.
Can't add song by Lil Baby & Gunna, artist must be Gunna.
id: 250301, artist:
{'artist': {'alternate_names': ['Martin Sue', 'Charlie Otto Puth', 'Charles Otto Puth Jr.'], 'api_path': '/artists/250301', 'description': {'plain': 'Charlie Puth is an American singer-songwriter who started by making and covering songs on his YouTube channel. His big break came when he was featured on the Furious 7 song “See You Again,” which he had written for a friend who passed away in a similar accident. The song reached #1 on the Billboard Hot 100 and stayed there for twelve weeks, becoming the longest-running rap number one along with Eminem’s “Lose Yourself.” The song’s iconic music video was also the most viewed video on all of YouTube for a sho

Can't add song by Lil Dicky, artist must be Charlie Puth.
Can't add song by The Kid LAROI & Justin Bieber, artist must be Charlie Puth.
Can't add song by Ariana Grande, artist must be Charlie Puth.
Can't add song by Justin Bieber, artist must be Charlie Puth.
Can't add song by Charlie Puth & Jung Kook, artist must be Charlie Puth.
Can't add song by Lil Wayne & Charlie Puth, artist must be Charlie Puth.
Can't add song by 5 Seconds of Summer, artist must be Charlie Puth.
Can't add song by Ava Max, artist must be Charlie Puth.
Can't add song by Trey Songz, artist must be Charlie Puth.
Can't add song by Katy Perry, artist must be Charlie Puth.
Can't add song by Flo Rida & 99 Percent, artist must be Charlie Puth.
We Don’t Talk Anymore already in Charlie Puth, not adding song.
Can't add song by Liam Payne, artist must be Charlie Puth.
Can't add song by Michael Bublé, artist must be Charlie Puth.
Can't add song by Faouzia & John Legend, artist must be Charlie Puth.
Can't add song by Katy Perr

Can't add song by SCH, artist must be Lacrim.
id: 26369, artist:
{'artist': {'alternate_names': [], 'api_path': '/artists/26369', 'description': {'plain': 'Tame Impala is Kevin Parker’s psychedelic rock project from Perth, Australia. Originally signed to Modular Recordings, they are now also with Interscope Records.\n\nThe band came to prominence in 2010 with the release of their debut album, Innerspeaker. Their name refers to the impala, a medium-sized antelope. They see themselves as:\n\nA steady flowing psychedelic groove rock band that emphasizes dream-like melody.\n\nHere is another description, previously attributed to their website:\n\nTame Impala is the movement in the Orion Nebula and the slime from a snail journeying across a footpath. To humans, however, Tame Impala is more of a ‘music ensemble,’ but its various other forms should not be disregarded (colour that humans can see is only a small part of the electromagnetic spectrum, the rest is just as important to other things

Can't add song by The Weeknd, artist must be Tame Impala.
Can't add song by Miguel, artist must be Tame Impala.
id: 1137969, artist:
{'artist': {'alternate_names': ['Young Nigga World Melly', 'Melly', 'J. Demons', 'Jamell Maurice Demons', 'YNW Melvin', 'Jamell Demons'], 'api_path': '/artists/1137969', 'description': {'plain': "Jamell Maurice Demons (born May 1, 1999), better known by his stage name YNW Melly, is an American rapper, singer, and songwriter who emerged from Gifford, Florida. YNW Melly is also co-founder of rap collective, YNW. His music often draws from his lived experiences, evidenced by his 2017 incarceration for weapons charges. Upon release, he began to gain popularity from his track “Murder On My Mind.” which he wrote while in jail. After the release of the track, he was once again arrested as the song violated his probation, and sent him back to prison.\n\nMelly’s career initially started to take off in 2018 when singles such as “Virtual (Blue Balenciagas)”, “Butter

In [ ]:
def id(song) -> int:
    return song._id

def name(song) -> str:
    return song.title

def artist(song) -> str:
    return song.artist

def artist_id(song) -> int:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('primary_artist', {}).get('id', None)

def language(song) -> str:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('language', None)

def lyrics(song) -> str:
    lyrics = Lyrics.lyrics(song._id)
    print(lyrics)
    lyrics = lyrics.split('\n')[1:]
    return "".join(lyrics)

def map_song(song) -> dict:
    return {
        'ID': id(song),
        'Name': name(song),
        'Artist': artist(song),
        'Artist ID': artist_id(song),
        'Language': language(song),
        'Lyrics': lyrics(song)
    }

songs = Songs.songs()
songs_dict = list(map(map_song, songs))

df = DataFrame(songs_dict)
df.to_csv('songs.csv')